# Hacking with nmap

Lets have some fun with nmap...

First some conventions, this notebook is using a Jupyter Notepook with an Python Kernel though most of the stuff we will be doing are bash commands. The Jupyter Puthon Kernel allows running of bash commands by prepending a exclemation point in front of the command or by prepending a code block with %%bash, so that is why there is a bang in fron of bash commands or %%bash in code blocks with bash commands.

So what is nmap, nmap is a flexible, extensible network scanning engine. Without even touching on the extensible bits nmap contains many sophisticated useful features. The extensibility parts makes nmap a more powerful scanning enging but extends it's capabilities in to the realm of vulnerability scanning, network toolkits and exploit delivery.

Right so first thing is RTFM which on UNIX systems is the venerable manpate.

In [3]:
!man -P cat nmap

NMAP(1)                      Nmap Reference Guide                      NMAP(1)

NAME
       nmap - Network exploration tool and security / port scanner

SYNOPSIS
       nmap [Scan Type...] [Options] {target specification}

DESCRIPTION
       Nmap (“Network Mapper”) is an open source tool for network exploration
       and security auditing. It was designed to rapidly scan large networks,
       although it works fine against single hosts. Nmap uses raw IP packets
       in novel ways to determine what hosts are available on the network,
       what services (application name and version) those hosts are offering,
       what operating systems (and OS versions) they are running, what type of
       packet filters/firewalls are in use, and dozens of other
       characteristics. While Nmap is commonly used for security audits, many
       systems and network administrators find it useful for routine tasks
       such as network inventory, managing service upgrade schedules, and
       mo

BTW about the command we just ran to dumpt the man pate into the note book, well the the -P option tells man to use formattting like cat would and just dump the page instead of paginating it for a terminsl.

What a wall of text, well with great flexibility comes great complexity.. But who reads man pages anyways, your question is probably on Stackoverflow already/... ;)

Seriously though man pages are a tremendous resource, and prehaps a value lost in today's world of Aglie development.

So ther is a lot of information about nmap out there and if your familiar with nmap and nse you might as well as stop reading now. The main objective here is to hack with something that is new and unfamilar. It is imposible to know everything about everything, however with a the right analytical skills today especially it give you the tools to travel light and get things done.

What I wanted to focus on here is the extensibility aspect, in that I am refering to NSE the nmap scripting engine...

NMap comes with a number of scrpts utilizing NSE lets take a look at one real quick...But it might help if we did this with a purpose in mind....

Lets supose we thought that developers might be using the builtin Python http-server module for development and that they are doing it in their development directory which is a git repository, they could be exposing credentials that would allow injection of malicious code in to a souce tree that could make it in to production, what a nice place to put a backdoor....

Right so how do we do this? Youre vaugly familiar with nmap, it's a port scanner that would be good for perhaps scanning for ports that have a HTTP server running. But how do we know if the server is a Python http-server? So dorking about with nmap and browsing on the interwebs you find out about this cool feature of nmap that allows scripting, and that there a lot of built in scripts in /usr/share/nmap/scripts . There is even one called http-headers that prints out HTTP headers of any HTTP server that nmap connects to
This one dumps the banner output from an number of network protocols....

Looks like a good start and it'ts easy enough to stand up a Python http-server inside a directory with a git repo to test our theory....

In [ ]:
%%bash
mkdir ~/hacking-with-nmap
cd ~/hacking-with-nmap
git init
python3 -m http.server

```
Serving HTTP on 0.0.0.0 port 8000 ...
```
 Yikes that is listening on all addresses including accessable from other machines on the network and possibly the internet.
 
 We better kill http-server and restart it, on the local loop back port.

In [ ]:
%%%bash
pkill -f "python3 -m http.server"
python3 -m http.server --bind 127.0.0.1 8000

Now we are ready to have go at the mython http-server module with the NMap NSE script http-headers

In [3]:
%%bash
nmap --script http-headers -p 80,8000,8080 127.0.0.1


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-27 22:22 EDT
Nmap scan report for localhost (127.0.0.1)
Host is up (0.00013s latency).
PORT     STATE  SERVICE
80/tcp   closed http
8000/tcp open   http-alt
| http-headers: 
|   Server: SimpleHTTP/0.6 Python/3.5.2
|   Date: Mon, 28 May 2018 02:22:13 GMT
|   Content-type: text/html; charset=utf-8
|   Content-Length: 542
|   
|_  (Request type: HEAD)
8080/tcp closed http-proxy

Nmap done: 1 IP address (1 host up) scanned in 0.28 seconds


Perfect the Python http-server populates the server field in the HTTP headers now we know how to identify potential targets.

Okay so how do we figure out how to find out if the server has a git repo inside and get the information. Uh I guess we could write a scraper but that is work...

I wonder are there any NSE scripts that do something like what we want?

In [1]:
%%%bash
ls /usr/share/nmap/scripts| grep git

http-git.nse
http-gitweb-projects-enum.nse


Hmm http-git looks interesting. Lets get some more information. From reading the man page we can use the switch --script-help to get more information on a script

In [2]:
!nmap --script-help http-git


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 00:32 EDT

http-git
Categories: default safe vuln
https://nmap.org/nsedoc/scripts/http-git.html
  Checks for a Git repository found in a website's document root
  /.git/<something>) and retrieves as much repo information as
  possible, including language/framework, remotes, last commit
  message, and repository description.


Perfect it does pretty much what we want, list test it out

In [3]:
!nmap --script http-git -p 8000 127.0.0.1


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 00:35 EDT
Nmap scan report for localhost (127.0.0.1)
Host is up (0.00014s latency).
PORT     STATE SERVICE
8000/tcp open  http-alt
| http-git: 
|   127.0.0.1:8000/.git/
|     Git repository found!
|_    Repository description: Unnamed repository; edit this file 'description' to name the...

Nmap done: 1 IP address (1 host up) scanned in 0.32 seconds


Okay so we know how to find the target servers and we know how to find the git repo if it is there. But how do we ty the two together?

Well we could parse the text output, but that seems like it would be a pain and be brittle. Hmm something from the manpage sticks out nmap can  generate XML output to a file with the -oX switch

```bash
 nmap --script http-git -p 8000 127.0.0.1 -oX -
 ```
```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE nmaprun>
<?xml-stylesheet href="file:///usr/bin/../share/nmap/nmap.xsl" type="text/xsl"?>
<!-- Nmap 7.40 scan initiated Mon May 28 00:42:10 2018 as: nmap -&#45;script http-git -p 8000 -oX - 127.0.0.1 -->
<nmaprun scanner="nmap" args="nmap -&#45;script http-git -p 8000 -oX - 127.0.0.1" start="1527482530" startstr="Mon May 28 00:42:10 2018" version="7.40" xmloutputversion="1.04">
<scaninfo type="connect" protocol="tcp" numservices="1" services="8000"/>
<verbose level="0"/>
<debugging level="0"/>
<host starttime="1527482530" endtime="1527482530"><status state="up" reason="conn-refused" reason_ttl="0"/>
<address addr="127.0.0.1" addrtype="ipv4"/>
<hostnames>
<hostname name="localhost" type="PTR"/>
</hostnames>
<ports><port protocol="tcp" portid="8000"><state state="open" reason="syn-ack" reason_ttl="0"/><service name="http-alt" method="table" conf="3"/><script id="http-git" output="&#xa;  127.0.0.1:8000/.git/&#xa;    Git repository found!&#xa;    Repository description: Unnamed repository; edit this file &apos;description&apos; to name the...&#xa;"><table key="127.0.0.1:8000/.git/">
<table key="files-found">
<elem key=".git/COMMIT_EDITMSG">false</elem>
<elem key=".git/description">true</elem>
<elem key=".git/config">true</elem>
<elem key=".git/info/exclude">true</elem>
<elem key=".gitignore">false</elem>
</table>
<elem key="repository-description">Unnamed repository; edit this file &apos;description&apos; to name the repository.&#xa;</elem>
</table>
</script></port>
</ports>
<times srtt="191" rttvar="3775" to="100000"/>
</host>
<runstats><finished time="1527482530" timestr="Mon May 28 00:42:10 2018" elapsed="0.39" summary="Nmap done at Mon May 28 00:42:10 2018; 1 IP address (1 host up) scanned in 0.39 seconds" exit="success"/><hosts up="1" down="0" total="1"/>
</runstats>
</nmaprun>
```

So that is nice, but, ug now I have to figure out how to deal with the XML (Note using the XML output is perfectly reasonable and good, just not one were going to take)

Okay so what if we could mash the two NSE scripts together and make one script that does what we want. Besides wouldn't it be cool if we could make our own NSE scripts that might be a nice think to know how to do.

The NMap NSE uses an ebeded Lua interperter for it's scrptting engine. Lua is an awesome little language similar to Javascript, small light and fast. In many ways a better Javascript than Javascript. The primary similiarities to javascript is the object systems, both Javascript and Lua use prototype based object systems.  That is to say new objects are built on top of existing objects which serve as ptototypes adding attributes and methods. There some other syntactic differences. But you  know what wether we know lua or not we only need to know enough to get this working, I'm sure the interwebs can help, enough.